In [1]:
import os

def get_unique_filename(directory, filename):
    base_name, extension = os.path.splitext(filename)
    counter = 1
    unique_filename = filename
    
    # 파일 이름이 이미 존재할 경우
    while os.path.exists(os.path.join(directory, unique_filename)):
        unique_filename = f"{base_name}_{counter}{extension}"
        counter += 1
    
    return unique_filename

# 데이터 저장 경로와 기본 파일 이름
output_dir = 'data'
output_file = 'store_data.csv'

# 디렉토리가 존재하지 않으면 생성
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 고유한 파일 이름 생성
unique_file = get_unique_filename(output_dir, output_file)

print(f"저장할 파일 이름: {unique_file}")

저장할 파일 이름: store_data_3.csv


In [2]:
import time
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

url = 'https://map.naver.com/v5/search'
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)

keywords = ['이마트', '롯데마트', '홈플러스', '코스트코']

def time_wait(num, code):
    try:
        wait = WebDriverWait(driver, num).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, code)))
    except:
        print(code, '태그를 찾지 못하였습니다.')
        driver.quit()
    return wait

def switch_frame(frame_id):
    try:
        wait = WebDriverWait(driver, 10).until(
            EC.frame_to_be_available_and_switch_to_it((By.ID, frame_id))
        )
    except:
        print(f"Frame {frame_id}를 찾을 수 없습니다.")
        driver.quit()

def page_down(num):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.click()
    for i in range(num):
        body.send_keys(Keys.PAGE_DOWN)

def clean_address(address):
    """주소의 끝 두 글자(복사)를 제거."""
    return address[:-2].strip() if len(address) > 2 else address        


def collect_data_for_keyword(keyword):
    driver.get(url)
    
    # (1) 검색창 찾기
    search = driver.find_element(By.CSS_SELECTOR, 'div.input_box > input.input_search')
    search.send_keys(keyword)  # 검색어 입력
    search.send_keys(Keys.ENTER)  # 엔터버튼 누르기

    time_wait(10, 'div.input_box > input.input_search')  # 검색 결과 대기
    switch_frame('searchIframe')

    page_down(40)
    sleep(3)
    
    data_list = []


    print(f'[크롤링 시작: {keyword}]')

    while True:
        # 데이터 추출
        parking_list = driver.find_elements(By.CSS_SELECTOR, 'li.QPJLF') #대형마트 리스트
        next_btn = driver.find_elements(By.CSS_SELECTOR, '.zRM9F > a') #다음버튼
        names = driver.find_elements(By.CSS_SELECTOR, '.EnJnK')  # (3) 장소명
        address_buttons = driver.find_elements(By.CSS_SELECTOR, '.lWwyx > a')

        if len(names) != len(address_buttons):
            print(f"경고: 대형마트 목록과 주소 버튼 목록의 길이가 일치하지 않습니다. (대형마트: {len(names)}, 주소 버튼: {len(address_buttons)})")

        for data in range(len(parking_list)):
            try:
                jibun_address = ''
                road_address = ''
                parking_name = names[data].text

                if data < len(address_buttons):
                    address_buttons[data].click()
                    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CSS_SELECTOR, '.AbTyi > div')))
                else:
                    print(f"경고: 주소 버튼이 부족합니다. (현재 인덱스: {data}, 버튼 수: {len(address_buttons)})")
                    continue

                addr = driver.find_elements(By.CSS_SELECTOR, '.AbTyi > div')

                if len(addr) == 1:
                    if addr[0].text.startswith('지번'):
                        jibun_address = clean_address(addr[0].text.split('\n')[0][2:])  # 복사 버튼, 우편번호 제외
                    elif addr[0].text.startswith('도로'):
                        road_address = clean_address(addr[0].text.split('\n')[0][3:])  # 복사 버튼, 우편번호 제외
                elif len(addr) == 2:
                    road_address = clean_address(addr[0].text[3:])
                    jibun_address = clean_address(addr[1].text[2:])

                data_list.append([parking_name if parking_name else '', road_address if road_address else '', jibun_address if jibun_address else ''])
                print(f'{parking_name} ...완료')

            except Exception as e:
                print(e)
                print('ERROR!' * 3)
                data_list.append([parking_name if parking_name else '', road_address if road_address else '', jibun_address if jibun_address else ''])
                print(f'{parking_name} ...완료')
                sleep(1)
        # 다음 페이지 버튼 누를  수 없으면 종료
        if not next_btn[-1].is_enabled() or next_btn[-1].get_attribute('aria-disabled') == 'true':
            break

        if names:  # 리스트가 비어있지 않은 경우에만 접근
            if names[-1]:  # 마지막 대형마트일 경우 다음 버튼 클릭
                next_btn[-1].click()
                sleep(2)
            else:
                print('페이지 인식 못함 또는 데이터 없음')
                break
        

    return data_list

In [3]:
# 크롤링 실행
all_data = []

for keyword in keywords:  
    keyword_data = collect_data_for_keyword(keyword)
    all_data.extend(keyword_data)

# 데이터를 DataFrame으로 변환하고 CSV 파일로 저장
df = pd.DataFrame(all_data, columns=['Name', 'Road Address', 'Jibun Address'])
df.to_csv(os.path.join(output_dir, unique_file), index=False, encoding='utf-8-sig')
print(f'모든 데이터가 {unique_file} 파일에 저장되었습니다.')

driver.quit()

[크롤링 시작: 이마트]
이마트 미아점 ...완료
이마트 하월곡점 ...완료
이마트 월계점 ...완료
이마트 청계천점 ...완료
이마트 묵동점 ...완료
이마트 창동점 ...완료
이마트 왕십리점 ...완료
이마트 은평점 ...완료
이마트 자양점 ...완료
이마트 마포점 ...완료
이마트 신촌점 ...완료
이마트 별내점 ...완료
이마트 용산점 ...완료
이마트 수색점 ...완료
이마트 다산점 ...완료
이마트 천호점 ...완료
이마트 역삼점 ...완료
이마트 여의도점 ...완료
이마트 명일점 ...완료
이마트 영등포점 ...완료
이마트 수서점 ...완료
이마트 목동점 ...완료
이마트 신도림점 ...완료
이마트 양재점 ...완료
이마트 의정부점 ...완료
이마트 화정점 ...완료
이마트 가든파이브점 ...완료
이마트 하남점 ...완료
이마트 구로점 ...완료
이마트 진접점 ...완료
이마트 신월점 ...완료
이마트 남양주점 ...완료
이마트 과천점 ...완료
이마트 일산점 ...완료
이마트 성남점 ...완료
이마트 풍산점 ...완료
이마트 광명소하점 ...완료
이마트 평촌점 ...완료
이마트 안양점 ...완료
이마트 부천점 ...완료
이마트 중동점 ...완료
이마트 양주점 ...완료
이마트 계양점 ...완료
이마트 파주운정점 ...완료


KeyboardInterrupt: 